# Natural language processing concepts with spaCy

By [Allison Parrish](http://www.decontextualize.com/)

“Natural Language Processing” is a field at the intersection of computer science, linguistics and artificial intelligence which aims to make the underlying structure of language available to computer programs for analysis and manipulation. It’s a vast and vibrant field with a long history! New research and techniques are being developed constantly.

The aim of this notebook is to introduce a few simple concepts and techniques from NLP—just the stuff that’ll help you do creative things quickly, and maybe open the door for you to understand more sophisticated NLP concepts that you might encounter elsewhere. We'll start with simple extraction tasks: isolating words, sentences, and parts of speech. By the end, we'll have a few working systems for creating sophisticated text generators that function by remixing texts based on their constituent linguistic units. This tutorial is written for Python 3.6+.

There are a number of libraries for performing natural language processing tasks in Python, including:

* [NLTK](http://www.nltk.org/): a workhorse of a library, still widely used, but somewhat dated in terms of capabilities
* [Stanza](https://stanfordnlp.github.io/stanza/index.html) is a newish natural language processing library for Python, developed by the storied [Stanford NLP Group](https://nlp.stanford.edu/). It includes a client library for Stanford NLP's [CoreNLP library](https://stanfordnlp.github.io/CoreNLP/) for functionality not yet natively available in Python.
* [AllenNLP](https://allennlp.org/) is a library for developing and deploying natural language processing machine learning models, including models for [sentiment analysis](https://demo.allennlp.org/sentiment-analysis), [constituency parsing](https://demo.allennlp.org/constituency-parsing) and [co-reference resolution](https://demo.allennlp.org/coreference-resolution).

But we'll be using a library called [spaCy](https://spacy.io/), which very powerful and easy for newcomers to understand. It's been among the most important tools in my text processing toolbox for many years!


## Natural language

“Natural language” is a loaded phrase: what makes one stretch of language “natural” while another stretch is not? NLP techniques are opinionated about what language is and how it works; as a consequence, you’ll sometimes find yourself having to conceptualize your text with uncomfortable abstractions in order to make it work with NLP. (This is especially true of poetry, which almost by definition breaks most “conventional” definitions of how language behaves and how it’s structured.)

Of course, a computer can never really fully “understand” human language. Even when the text you’re using fits the abstractions of NLP perfectly, the results of NLP analysis are always going to be at least a little bit inaccurate. But often even inaccurate results can be “good enough”—and in any case, inaccurate output from NLP procedures can be an excellent source of the sublime and absurd juxtapositions that we (as poets) are constantly in search of.

## Language support

Historically, most NLP researchers have focused their efforts on English specifically. But many natural language processing libraries now support a wide range of languages. You can find the full [list of supported languages](https://spacy.io/usage/models#languages) on their website, though the robustness of these models varies from one language to the next, as does the specifics of how the model works. (For example, different languages have different ideas about what a "part of speech" is.) The examples in this notebook are primarily in English. If you're having trouble applying these techniques to other languages, send me an e-mail—I'd be happy to help you figure out how to get things working for languages other than English!

## English grammar: a crash course

The only thing I believe about English grammar is [this](http://www.writing.upenn.edu/~afilreis/88v/creeley-on-sentence.html):

> "Oh yes, the sentence," Creeley once told the critic Burton Hatlen, "that's
> what we call it when we put someone in jail."

There is no such thing as a sentence, or a phrase, or a part of speech, or even
a "word"---these are all pareidolic fantasies occasioned by glints of sunlight
we see on reflected on the surface of the ocean of language; fantasies that we
comfort ourselves with when faced with language's infinite and unknowable
variability.

Regardless, we may find it occasionally helpful to think about language using
these abstractions. The following is a gross oversimplification of both how
English grammar works, and how theories of English grammar work in the context
of NLP. But it should be enough to get us going!

### Sentences and parts of speech

English texts can roughly be divided into "sentences." Sentences are themselves
composed of individual words, each of which has a function in expressing the
meaning of the sentence. The function of a word in a sentence is called its
"part of speech"—i.e., a word functions as a noun, a verb, an adjective, etc.
Here's a sentence, with words marked for their part of speech:

    I       really love entrees       from        the        new       cafeteria.
    pronoun adverb verb noun (plural) preposition determiner adjective noun

Of course, the "part of speech" of a word isn't a property of the word itself.
We know this because a single "word" can function as two different parts of speech:

> I love cheese.

The word "love" here is a verb. But here:

> Love is a battlefield.

... it's a noun. For this reason (and others), it's difficult for computers to
accurately determine the part of speech for a word in a sentence. (It's
difficult sometimes even for humans to do this.) But NLP procedures do their
best!

### Phrases and larger syntactic structures

There are several different ways for talking about larger syntactic structures in sentences. The scheme used by spaCy is called a "dependency grammar." We'll talk about the details of this below.

## Installing spaCy

There are [instructions for installing spaCy](https://spacy.io/usage) on the spaCy web page. You can also install it by running the following cell in this notebook:

In [1]:
import sys
!conda install -c conda-forge -y --prefix {sys.prefix} spacy

Solving environment: done

## Package Plan ##

  environment location: /Users/gabriel/opt/anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    catalogue-2.0.8            |   py39h6e9494a_1          32 KB  conda-forge
    commonmark-0.9.1           |             py_0          46 KB  conda-forge
    conda-23.1.0               |   py39h6e9494a_0         908 KB  conda-forge
    confection-0.0.4           |   py39h2a79d86_1          61 KB  conda-forge
    cymem-2.0.7                |   py39h7a8716b_1          35 KB  conda-forge
    cython-blis-0.7.9          |   py39h7cc1f47_1         5.6 MB  conda-forge
    langcodes-3.3.0            |     pyhd8ed1ab_0         156 KB  conda-forge
    murmurhash-1.0.9           |   py39h7a8716b_1          23 KB  conda-forge
    pathy-0.10.1               |     pyhd8ed1ab_0          42 KB  conda-forge
    preshed-3.

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


You'll also need to download a language model. You can download the default language model for English by running the cell below:

In [2]:
import sys
!{sys.executable} -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


Replace `en_core_web_md` with the name of the model you want to install. [The spaCy documentation explains the difference between the various models](https://spacy.io/models).

The language model contains machine learning models for splitting texts into sentences and words, tagging words with their parts of speech, identifying entities, and discovering the syntactic structure of sentences.

## Basic usage

Import `spacy` like any other Python module:

In [1]:
import spacy

Create a new spaCy object using `spacy.load(...)`. The name in the parentheses is the same as the name of the model you downloaded above. If you downloaded a different model, you can put its name here instead.

In [2]:
nlp = spacy.load('en_core_web_md')

It's more fun doing natural language processing on text that you're interested in. I recommend grabbing a something from [Project Gutenberg](http://www.gutenberg.org/). Download a plain text file and put it in the same directory as this notebook, taking care to replace the filename in the cell below with the name of the file you downloaded.

In [3]:
# replace "84-0.txt" with the name of your own text file
text = open("allbills.txt").read()

Now, use spaCy to parse it. (This might take a while, depending on the size of your text.)

In [6]:
nlp.max_length = 5000000 #or any large value, as long as you don't run out of RAM
doc = nlp(text, disable = ['ner', 'parser'])

Right off the bat, the spaCy library gives us access to a number of interesting units of text:

* All of the sentences (`doc.sents`)
* All of the words (`doc`)
* All of the "named entities," like names of places, people, #brands, etc. (`doc.ents`)
* All of the "noun chunks," i.e., nouns in the text plus surrounding matter like adjectives and articles

The cell below, we extract these into variables so we can play around with them a little bit.

In [8]:
nlp.add_pipe('sentencizer')

In [9]:
sentences = list(doc.sents)
words = [w for w in list(doc) if w.is_alpha]
noun_chunks = list(doc.noun_chunks)
entities = list(doc.ents)

ValueError: [E030] Sentence boundaries unset. You can add the 'sentencizer' component to the pipeline with: `nlp.add_pipe('sentencizer')`. Alternatively, add the dependency parser or sentence recognizer, or set sentence boundaries by setting `doc[i].is_sent_start`.

### Counting and sampling

With this information in hand, we can answer interesting questions like: how many sentences are in the text?

In [20]:
len(sentences)

2870

Using `random.sample()`, we can get a small, randomly-selected sample from these lists. Here are five random sentences:

In [29]:
import random
for item in random.sample(sentences, 5):
    print(item.text.strip().replace("\n", " "))
    print()

— 15 ‘‘(I) I N GENERAL .—The term 16 ‘gender transition surgery’ means any 17 surgical service, including genital or 18 non-genital surgery, performed for the 19 purpose of assisting an individual with 20 a gender transition.

(a)

o

NEW LAW

Performing sterilization surgeries , including castration, vasectomy, hystere ctomy, oophorectomy, orchiectomy, and penectomy ; 5.



Ten random words:

In [30]:
for item in random.sample(words, 10):
    print(item.text)

Surgical
inpatient
investigating
the
the
biologically
result
setting
synt
with


Ten random noun chunks:

In [34]:
for item in random.sample(noun_chunks, 10):
    print(item.text)

the Req
11 (2) "Cross-sex hormones
unprofessional conduct
gender
this section
AMENDATORY
23 secondary associations
the ova
this chapter
o


Ten random entities:

In [41]:
for item in random.sample(entities, 10):
    print(item.text)

under 30
12
3
26
9
23
22
3
34
more than 2


### spaCy data types

Note that the values that spaCy returns belong to specific spaCy data types. You can read more about [these data types](https://spacy.io/api) in the spaCy documentation, in particular [spans](https://spacy.io/api/span/) and [tokens](https://spacy.io/api/token). (Spans represent sequences of tokens; a sentence in spaCy is a span, and a word is a token.) If you want a list of strings instead of a list of spaCy objects, use the `.text` attribute, which works for spans and tokens alike. For example:

In [42]:
sentence_strs = [item.text for item in doc.sents]

In [45]:
random.sample(sentence_strs, 10)

['A new section of law not to be codified in the Oklahoma Statutes reads as follows: This act shall be known and may be cited as the "Women\'s Bi ll of Rights Act of 2023 ".',
 'Money Appropriated in this Bill: 20 None 21 Other Special Clauses:',
 '6 Sec.',
 'By: Jett AS INTRODUCED An Act relating to health care; creating the Oklahoma Save Adolescents fro m Experimentatio n (SAFE) Act ; providing short title; defining terms ; prohibiting provision of or referral for gender transition procedures ; providing exceptions; pro hibiting certain uses of public funds, public facilities , public employees , and state M edicaid program ; providing for administrative and civil enforce ment; au thorizing certain civil actions and relief ; providing certain construction; providing for codification; and declaring an emergency .',
 'BE IT ENACTED BY THE PEOPLE OF T HE STATE OF OKLAHOMA: SECTION 1.',
 '2. Disciplinary proceedings against the physician or other health care professional must be commence

### Parts of speech

The spaCy parser allows us to check what part of speech a word belongs to. In the cell below, we create four different lists—`nouns`, `verbs`, `adjs` and `advs`—that contain only words of the specified parts of speech. ([There's a full list of part of speech tags here](https://spacy.io/docs/usage/pos-tagging#pos-tagging-english)).

In [46]:
nouns = [w for w in words if w.pos_ == "NOUN"]
verbs = [w for w in words if w.pos_ == "VERB"]
adjs = [w for w in words if w.pos_ == "ADJ"]
advs = [w for w in words if w.pos_ == "ADV"]

And now we can print out a random sample of any of these:

In [51]:
for item in random.sample(nouns, 20): # change "nouns" to "verbs" or "adjs" or "advs" to sample from those lists!
    print(item.text)

extent
sex
state
retaliation
procedures
wardship
letters
grounds
sex
undervirilization
relationship
sex
adolescence
years
person
action
treatment
bill
patients
statement


### Entity types

The parser in spaCy not only identifies "entities" but also assigns them to a particular type. [See a full list of entity types here.](https://spacy.io/docs/usage/entity-recognition#entity-types) Using this information, the following cell builds lists of the people, locations, and times mentioned in the text:

In [52]:
people = [e for e in entities if e.label_ == "PERSON"]
locations = [e for e in entities if e.label_ == "LOC"]
times = [e for e in entities if e.label_ == "TIME"]

And then you can print out a random sample:

In [60]:
for item in random.sample(times, 20): # change "times" to "people" or "locations" to sample those lists
    print(item.text.strip())

2/7/2023 4:15 PM
02:50 PM
10:53 AM Jacketed
10:53 AM
2:09 PM
2/7/2023 4:15 PM
10:53 AM
2/7/2023 4:15 PM
2/7/2023 4:15 PM
10:53 AM Jacketed
12/9/2022 12:25
2/7/2023 4:15 PM Jacketed
2/7/2023 4:15 PM
2/7/2023 4:15 PM
10:53 AM Jacketed
2/7/2023 4:15 PM
9:14 AM - 2
11:25:39
2:09 PM
12/9/2022 12:25


## Finding the most common

After we've parsed the text out into meaningful units, it might be interesting to see which examples of those units are the most common in a text.

One of the most common tasks in text analysis is counting how many times things occur in a text. The easiest way to do this in Python is with the [`Counter` object, contained in the `collections` module](https://docs.python.org/3/library/collections.html#collections.Counter). Run the following cell to create a `Counter` object to count your words.

In [61]:
from collections import Counter
word_count = Counter([w.text for w in words])

Once you've created the counter, you can check to see how many times any word occurs like so:

In [64]:
word_count['gender']

749

The `Counter` object's `.most_common()` method gives you access to a list of tuples with words and their counts, sorted in reverse order by count:

In [65]:
word_count.most_common(10)

[('or', 4167),
 ('the', 3842),
 ('of', 3642),
 ('a', 2282),
 ('to', 2234),
 ('and', 1898),
 ('in', 1583),
 ('for', 1023),
 ('that', 1002),
 ('sex', 989)]

The code in the following cell prints this out nicely:

In [66]:
for word, count in word_count.most_common(20):
    print(word, count)

or 4167
the 3842
of 3642
a 2282
to 2234
and 1898
in 1583
for 1023
that 1002
sex 989
is 851
with 829
as 802
any 787
an 757
gender 749
by 717
individual 689
not 658
this 652


You'll note that the list of most frequent words here likely reflects the overall frequency of words in English. Consult my [Quick and dirty keywords](quick-and-dirty-keywords.ipynb) tutorial for some simple strategies for extracting words that are most unique to a text (rather than simply the most frequent words). You may also consider [removing stop words](https://stackabuse.com/removing-stop-words-from-strings-in-python/) from the list.

### Writing to a file

You might want to export lists of words or other things that you make with spaCy to a file, so that you can bring them into other Python programs (or just other programs that form a part of your workflow). One way to do this is to write each item to a single line in a text file. The code in the following cell does exactly this for the word list that we just created:

In [23]:
with open("words.txt", "w") as fh:
    fh.write("\n".join([w.text for w in words]))

The following cell defines a function that performs this for any list of spaCy values you pass to it:

In [24]:
def save_spacy_list(filename, t):
    with open(filename, "w") as fh:
        fh.write("\n".join([item.text for item in t]))

Here's how to use it:

In [25]:
save_spacy_list("words.txt", words)

Since we're working with Counter objects a bunch in this notebook, it makes sense to find a way to save these as files too. The following cell defines a function for writing data from a `Counter` object to a file. The file is in "tab-separated values" format, which you can open using most spreadsheet programs. Execute it before you continue:

In [26]:
def save_counter_tsv(filename, counter, limit=1000):
    with open(filename, "w") as outfile:
        outfile.write("key\tvalue\n")
        for item, count in counter.most_common():
            outfile.write(item.strip() + "\t" + str(count) + "\n")    

Now, run the following cell. You'll end up with a file in the same directory as this notebook called `100_common_words.tsv` that has two columns, one for the words and one for their associated counts:

In [27]:
save_counter_tsv("100_common_words.tsv", word_count, 100)

Try opening this file in Excel or Google Docs or Numbers!

If you want to write the data from another `Counter` object to a file:

* Change the filename to whatever you want (though you should probably keep the `.tsv` extension)
* Replace `word_count` with the name of any of the `Counter` objects we've made in this sheet and use it in place of `word_count`
* Change the number to the number of rows you want to include in your spreadsheet.

### When do things happen in this text?

Here's another example. Using the `times` entities, we can make a spreadsheet of how often particular "times" (durations, times of day, etc.) are mentioned in the text.

In [28]:
time_counter = Counter([e.text.lower().strip() for e in times])
save_counter_tsv("time_count.tsv", time_counter, 100)

Do the same thing, but with people:

In [29]:
people_counter = Counter([e.text.lower() for e in people])
save_counter_tsv("people_count.tsv", people_counter, 100)

## More about words

The list of words that we made above is actually a list of spaCy [Token](https://spacy.io/docs/api/token) objects, which have several interesting attributes. The `.text` attribute gives the  text of the word (as a Python string), and the `.lemma_` attribute gives the word's "lemma" (explained below):

In [73]:
for word in random.sample(words, 12):
    print(word.text, "→", word.lemma_)

as → as
sex → sex
registered → register
fact → fact
AS → as
information → information
unprofessional → unprofessional
that → that
Child → Child
procedures → procedure
copy → copy
as → as


A word's "lemma" is its most "basic" form, the form without any morphology
applied to it. "Sing," "sang," "singing," are all different "forms" of the
lemma *sing*. Likewise, "octopi" is the plural of "octopus"; the "lemma" of
"octopi" is *octopus*.

"Lemmatizing" a text is the process of going through the text and replacing
each word with its lemma. This is often done in an attempt to reduce a text
to its most "essential" meaning, by eliminating pesky things like verb tense
and noun number.

Individual sentences can also be iterated over to get a list of words in that sentence:

In [74]:
sentence = random.choice(sentences)
for word in sentence:
    print(word.text)

17
(
2
)
Nothing
in
this
section
may
be
construed
to
prevent
a
carrier
18
from
appropriately
utilizing
reasonable
medical
management
19
techniques
.


### Parts of speech

Token objects are tagged with their part of speech. For whatever reason, spaCy gives you this part of speech information in two different formats. The `pos_` attribute gives the part of speech using the [universal POS tag](https://universaldependencies.org/u/pos/) system, while the `tag_` attribute gives a more specific designation, using the [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) system. (Models for different languages will use different schemes; consult the [documentation for your model](https://spacy.io/models) for more information). We used this attribute earlier in the notebook to extract lists of words that had particular parts of speech, but you can access the attribute in other contexts as well:

In [75]:
for item in random.sample(words, 24):
    print(item.text, "/", item.pos_, "/", item.tag_)

udicial / ADJ / JJ
local / ADJ / JJ
General / PROPN / NNP
penectomy / NOUN / NN
patient / ADJ / JJ
of / ADP / IN
or / CCONJ / CC
either / CCONJ / CC
examination / NOUN / NN
intended / VERB / VBN
to / ADP / IN
benefit / NOUN / NN
for / ADP / IN
with / ADP / IN
Class / PROPN / NNP
that / PRON / WDT
female / ADJ / JJ
Remove / VERB / VB
and / CCONJ / CC
biological / ADJ / JJ
action / NOUN / NN
by / ADP / IN
sex / NOUN / NN
federal / ADJ / JJ


The `spacy.explain()` function also gives information about what part of speech tags mean:

In [76]:
spacy.explain('VBP')

'verb, non-3rd person singular present'

### Specific forms with `.tag_`

The `.pos_` attribute only gives us general information about the part of speech. The `.tag_` attribute allows us to be more specific about the kinds of verbs we want. For example, this code gives us only the verbs in past participle form:

In [77]:
only_past = [item.text for item in doc if item.tag_ == 'VBN']

In [87]:
random.sample(only_past, 12)

['known',
 'submitted',
 'meant',
 'based',
 'specified',
 'caused',
 'given',
 'certified',
 'situated',
 'disciplined',
 'caused',
 'limited']

Or only plural nouns:

In [88]:
only_plural = [item.text for item in doc if item.tag_ == 'NNS']

In [94]:
random.sample(only_plural, 12)

['increases',
 'procedures',
 'females',
 'children',
 'activities',
 'recommendations',
 'procedures',
 'religions',
 'procedures',
 'surgeries',
 'options',
 'features']

## Larger syntactic units

Okay, so we can get individual words and small phrases, like named entities and noun chunks. Great! But what if we want larger chunks, based on their syntactic role in the sentence? For this, we'll need to learn about how spaCy parses sentences into its syntactic components.

### Understanding dependency grammars

The spaCy library parses the underlying sentences using a [dependency grammar](https://en.wikipedia.org/wiki/Dependency_grammar). Dependency grammars look different from the kinds of sentence diagramming you may have done in high school, and even from tree-based [phrase structure grammars](https://en.wikipedia.org/wiki/Phrase_structure_grammar) commonly used in descriptive linguistics. The idea of a dependency grammar is that every word in a sentence is a "dependent" of some other word, which is that word's "head." Those "head" words are in turn dependents of other words. The finite verb in the sentence is the ultimate "head" of the sentence, and is not itself dependent on any other word. The dependents of a particular head are sometimes called its "children."

The question of how to know what constitutes a "head" and a "dependent" is complicated. For more details, consult [Dependency Grammar and Dependency Parsing](http://stp.lingfil.uu.se/~nivre/docs/05133.pdf). But here are some simple guidelines:

* A head determines the syntactic category of a phrase as a whole, and can often stand in for the phrase.
* The meaning of a phrase comes primarily from its head. The dependents further specify the meaning, but don't determine it.
* Heads are obligatory, while dependents are often optional.

For example, in the sentence "Large contented bears hibernate peacefully," *bears* is the head (a noun in this case) and *large* and *contented* are dependents (adjectives). The head of the phrase *large contented bears* is a noun, so the entire phrase is a noun. You could also rewrite the sentence to omit the dependents altogether, and it would still make sense: "Bears hibernate peacefully." Likewise, the adverb *peacefully* is a dependent of the head *hibernate*; the sentence could be rewritten as simply "Bears hibernate." 

Dependents are related to their heads by a *syntactic relation*. The name of the syntactic relation describes the relationship between the head and the dependent. Every token object in a spaCy document or sentence has attributes that tell you what the word's head is, what the dependency relationship is between that word and its head, and a list of that word's children (dependents).

The developers of spaCy included a little tool for visualizing the dependency relations of a particular sentence. Let's look at the sentence "I have eaten the plums that were in the icebox" as an example:

In [95]:
spacy.displacy.render(nlp("I have eaten the plums that were in the icebox."), style='dep')

The arcs you see originate at a head and terminate at dependencies (or children). If you follow all of the arcs back from dependent to head, you'll eventually get back to *eaten*, which is the *root* of the sentence. Each arc is labelled with the dependency *relation*, which tells us what role the dependent fills in the syntax and meaning of the parent word. For example, *I* is related to *eaten* by the `nsubj` relation, which means that *I* is the "nominal subject" of the verb. The word *icebox* is related to the head *in* via the `pobj` relation, meaning that *icebox* is the object of the preposition *in* An exhaustive list of the meanings of these relations can be found in the [Stanford Dependencies Manual](https://downloads.cs.stanford.edu/nlp/software/dependencies_manual.pdf).

The following code prints out each word in the sentence, the tag, the word's head, the word's dependency relation with its head, and the word's children (i.e., dependent words). (This code isn't especially useful on its own, it's just here to help show you how this functionality works.)

In [99]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text)
    print("Tag:", word.tag_)
    print("Head:", word.head.text)
    print("Dependency relation:", word.dep_)
    print("Children:", list(word.children))

Original sentence: Any school district, or public school thereof, and any state agency, department or office or political subdivision that collects vital statistics for the purpose of complying with anti-discrimination laws or for the purpose of gathering accurate public health, crime, economic or other data shall identify each individual who is part of the collected data set as either male or female at birth.

Word: Any
Tag: DT
Head: district
Dependency relation: det
Children: []

Word: school
Tag: NN
Head: district
Dependency relation: compound
Children: []

Word: district
Tag: NN
Head: identify
Dependency relation: nsubj
Children: [Any, school, ,, or, school]

Word: ,
Tag: ,
Head: district
Dependency relation: punct
Children: []

Word: or
Tag: CC
Head: district
Dependency relation: cc
Children: []

Word: public
Tag: JJ
Head: school
Dependency relation: amod
Children: []

Word: school
Tag: NN
Head: district
Dependency relation: conj
Children: [public, thereof, ,, and]

Word: thereof


Here's a list of a few dependency relations and what they mean, for quick reference:

* `nsubj`: this word's head is a verb, and this word is itself the subject of the verb
* `nsubjpass`: same as above, but for subjects in sentences in the passive voice
* `dobj`: this word's head is a verb, and this word is itself the direct object of the verb
* `iobj`: same as above, but indirect object
* `aux`: this word's head is a verb, and this word is an "auxiliary" verb (like "have", "will", "be")
* `attr`: this word's head is a copula (like "to be"), and this is the description attributed to the subject of the sentence (e.g., in "This product is a global brand", `brand` is dependent on `is` with the `attr` dependency relation)
* `det`: this word's head is a noun, and this word is a determiner of that noun (like "the," "this," etc.)
* `amod`: this word's head is a noun, and this word is an adjective describing that noun
* `prep`: this word is a preposition that modifies its head
* `pobj`: this word is a dependent (object) of a preposition

### Using .subtree for extracting syntactic units

That's all pretty abstract, so let's get a bit more concrete, and write some code that will let us extract syntactic units based on their dependency relation. There are a couple of things we need in order to do this. The `.subtree` attribute I used in the code above evaluates to a generator that can be flatted by passing it to `list()`. This is a list of the word's syntactic dependents—essentially, the "clause" that the word belongs to.

This function merges a subtree and returns a string with the text of the words contained in it:

In [100]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

With this function in our toolbox, we can write a loop that prints out the subtree for each word in a sentence. (Again, this code is just here to demonstrate what the process of grabbing subtrees looks like—it doesn't do anything useful yet!)

In [101]:
sent = random.choice(sentences)
print("Original sentence:", sent.text.replace("\n", " "))
for word in sent:
    print()
    print("Word:", word.text.replace("\n", " "))
    print("Flattened subtree: ", flatten_subtree(word.subtree).replace("\n", " "))

Original sentence: Sections 191.1720 to 191.1745 shall be known 1 and may be cited as the "Missouri Child and Adolescent 2 Protection (MCAP) Act".

Word: Sections
Flattened subtree:  Sections 191.1720 to 191.1745

Word: 191.1720
Flattened subtree:  191.1720

Word: to
Flattened subtree:  to 191.1745

Word: 191.1745
Flattened subtree:  191.1745

Word: shall
Flattened subtree:  shall

Word: be
Flattened subtree:  be

Word: known
Flattened subtree:  Sections 191.1720 to 191.1745 shall be known 1 and may be cited as the "Missouri Child and Adolescent 2 Protection (MCAP) Act".

Word: 1
Flattened subtree:  1

Word: and
Flattened subtree:  and

Word: may
Flattened subtree:  may

Word: be
Flattened subtree:  be

Word: cited
Flattened subtree:  may be cited as the "Missouri Child and Adolescent 2 Protection (MCAP) Act"

Word: as
Flattened subtree:  as the "Missouri Child and Adolescent 2 Protection (MCAP) Act"

Word: the
Flattened subtree:  the

Word: "
Flattened subtree:  "

Word: Missouri
Flat

Using the subtree and our knowledge of dependency relation types, we can write code that extracts larger syntactic units based on their relationship with the rest of the sentence. For example, to get all of the noun phrases that are subjects of a verb:

In [102]:
subjects = []
for word in doc:
    if word.dep_ in ('nsubj', 'nsubjpass'):
        subjects.append(flatten_subtree(word.subtree))

In [105]:
random.sample(subjects, 12)

['Gender',
 'Any referral for or provision of gender-transition procedures to an individual under twenty-six years of age',
 'LS 7276/DI 147',
 'surgery',
 '141 [(19)',
 'that',
 '6 not funding for the gender transition procedure',
 '2 (d) Health carriers',
 'testosterone or other androgens given to biological females in amounts that are larger or more potent',
 'Nothing in this subsection',
 "physician's services, inpatient and 41 outpatient hospital services, or prescribed drugs,",
 'Gender transition procedure"']

Or every prepositional phrase:

In [106]:
prep_phrases = []
for word in doc:
    if word.dep_ == 'prep':
        prep_phrases.append(flatten_subtree(word.subtree).replace("\n", " "))

In [107]:
random.sample(prep_phrases, 12)

['upon reaching the age of majority at any time from 12 that point until twenty years after reaching the age of 13 majority or four years from the time of discovery by the 14 injured party of both the injury and the causal relationship 15 between the treatment and the injury',
 'in athletic activities based on biological sex',
 'of this title',
 'for minors',
 'despite the lack of any long-term longitudinal studies evaluating the risks and benefits of using these drugs for the treatment of such distress or gender transition;(7',
 'under 97 chapter 334;',
 'of the violation',
 'on school -sponsored',
 'of a major bodily function',
 'for health care coverage that includes coverage for gender transition procedures',
 '-4',
 "including without limitation physician's services, inpatient and outpatient hospital services, or prescribed drugs related to gender transition that seeks to: (1) Alter or remove physical or anatomical characteristics or features that are typical for the individual’s 

## Generating text from extracted units

One thing I like to do is put together text from parts we've disarticulated with spaCy. Let's use Tracery to do this. If you don't know how to use Tracery, feel free to consult [my Tracery tutorial](tracery-and-python.ipynb) before continuing.

So I want to generate sentences based on things that I've extracted from my text. My first idea: get subjects of sentences, verbs of sentences, nouns and adjectives, and prepositional phrases:

In [108]:
subjects = [flatten_subtree(word.subtree).replace("\n", " ")
            for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]
past_tense_verbs = [word.text for word in words if word.tag_ == 'VBD' and word.lemma_ != 'be']
adjectives = [word.text for word in words if word.tag_.startswith('JJ')]
nouns = [word.text for word in words if word.tag_.startswith('NN')]
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ")
                for word in doc if word.dep_ == 'prep']

Notes on the code above:

* The `.replace("\n", " ")` is in there because spaCy treats linebreaks as normal whitespace, and retains them when we ask for the span's text. For formatting reasons, we want to get rid of this.
* I'm using `.startswith()` in the checks for parts of speech in order to capture other related parts of speech (e.g., `JJR` is comparative adjectives, `NNS` is plural nouns).
* I use only past tense verbs so we don't have to worry about subject/verb agreement in English. I'm excluding forms of *to be* because it is the only verb that agrees with its subject in the past tense.

Now I'll import Tracery. If you haven't already installed it, you can do so using the following cell:

In [109]:
import sys
!{sys.executable} -m pip install tracery

  Preparing metadata (setup.py) ... done
  Created wheel for tracery: filename=tracery-0.1.1-py3-none-any.whl size=7682 sha256=3c9c885c25a2d1eb3778c8ed75cef5efeb2e71bb421c7ff499ea66b517173523
  Stored in directory: /Users/gabriel/Library/Caches/pip/wheels/a3/74/46/553a949119086ada4c0ecf608890fb79513349b9057e514885
Successfully built tracery


In [110]:
import tracery
from tracery.modifiers import base_english

... and define a grammar. The "trick" of this example is that I grab entire rule expansions from the units extracted from the text using spaCy. The grammar itself is built around producing sentences that look and feel like English.

In [126]:
rules = {
    "origin": [
        "#subject.capitalize# #predicate#.",
        "#subject.capitalize# #predicate#.",
        "#prepphrase.capitalize#, #subject# #predicate#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "subject": subjects,
    "verb": past_tense_verbs,
    "noun": nouns,
    "adj": adjectives,
    "prepphrase": prep_phrases
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'That performed the program.'

Let's generate a whole paragraph of this and format it nicely:

In [127]:
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(12)])
print(fill(output, 60))

The individual ENACTED of a penis-like and scrotum-like
structure. Studies managed a treat. This act occurred the
features that afforded the diseased health. That found of
this section. The Code of Virginia comprised under 16
subsection (a). Relief found an Association. Which nated
with a medically verifiable disorder of sex development
including,. 12 (d) Remedies available to a parent under this
subsection ENACTED an athletic UNOFFICIAL. Of the body,
Unprofessional conduct ruled in Subsection . For purposes of
sections 191.1725 to 191.1745, both ovarian and testicular
tissue; or 177 (E) increased the care with. To persons born
with a medically verifiable disorder of sex devel opment,
including a p erson with external biological sex
characterist ics that a re irresolvabl y ambiguous , such as
thos e born with 4 6 XX Req, i ENACTED the e and the
disorder. At the time the individual abused, that rejected.


I like this approach for a number of reasons. Because I'm using a hand-written grammar, I have a great deal of control over the shape and rhythm of the sentences that are generated. But spaCy lets me pre-populate my grammar's vocabulary without having to write each item by hand.

## Further reading and resources

We've barely scratched the surface of what it's possible to do with spaCy. [The official site has a good list of guides to various natural language processing tasks](https://spacy.io/usage) that you should check out, and there are also [a handful of books that dig deeper into using spaCy for natural language processing tasks](https://spacy.io/universe/category/books).